In [ ]:
# Import required libraries
import numpy_financial as npf
import pandas as pd
import numpy as np

# Import the modular chit fund calculator
from chit_fund_calculator import (
    ChitFundConfig,
    ChitFundCalculator,
    ScenarioAnalyzer,
    ReportGenerator,
    ChitFrequency
)

In [2]:
total_installments = 14
current_installment_number = 4
full_chit_value = 700000
chit_frequency_per_year = 2
previous_installments = [42000, 40000, 40000, 43000]
bid_amount = 100000


def get_prize_amount(full_chit_value, bid_amount):
    return full_chit_value - bid_amount


def create_cashflows(
    previous_installments,
    prize_amount,
    winner_installment_amount,
    current_installments_number,
    total_installments,
):
    future_winner_installments = [-winner_installment_amount] * (
        total_installments - current_installments_number
    )
    cashflows = (
        [i * -1 for i in previous_installments]
        + [prize_amount]
        + future_winner_installments
    )
    return cashflows


def calculate_annual_irr(cashflows, chit_frequency_per_year):
    irr_per_period = npf.irr(cashflows)
    annual_irr = ((1 + irr_per_period) ** chit_frequency_per_year) - 1
    return annual_irr


def analyze_bid_scenarios(
    total_installments,
    current_installment_number,
    full_chit_value,
    chit_frequency_per_year,
    previous_installments,
    bid_amounts_to_test=None,
    winner_installment_amount=50000,
):
    """
    Analyze multiple bid amount scenarios to compare IRR and total costs.

    Parameters:
    - total_installments: Total number of installments in the chit
    - current_installment_number: The installment number when winning
    - full_chit_value: Total chit fund value
    - chit_frequency_per_year: Payment frequency (e.g., 2 for semi-annual)
    - previous_installments: List of previous installment amounts paid
    - bid_amounts_to_test: List of bid amounts to analyze (optional)
    - winner_installment_amount: Fixed amount winner pays after winning

    Returns:
    - DataFrame with scenario analysis results
    """

    if bid_amounts_to_test is None:
        # Default: Test bid amounts from 50k to 200k in 10k increments
        bid_amounts_to_test = range(50000, 210000, 10000)

    results = []

    for bid_amount in bid_amounts_to_test:
        # Reuse existing functions
        prize_amount = get_prize_amount(full_chit_value, bid_amount)
        cashflows = create_cashflows(
            previous_installments,
            prize_amount,
            winner_installment_amount,
            current_installment_number,
            total_installments,
        )

        try:
            annual_irr = calculate_annual_irr(cashflows, chit_frequency_per_year)

            # Calculate total repayment
            total_paid_before = sum(previous_installments)
            total_paid_after = winner_installment_amount * (
                total_installments - current_installment_number
            )
            total_repayment = total_paid_before + total_paid_after

            # Calculate net interest cost
            net_interest_cost = total_repayment - prize_amount

            results.append(
                {
                    "Bid Amount": bid_amount,
                    "Prize Received": prize_amount,
                    "Total Repayment": total_repayment,
                    "Net Interest Cost": net_interest_cost,
                    "Annual IRR (%)": annual_irr * 100,
                    "Effective Interest (%)": (net_interest_cost / prize_amount) * 100,
                }
            )
        except:
            # Handle cases where IRR cannot be calculated
            results.append(
                {
                    "Bid Amount": bid_amount,
                    "Prize Received": prize_amount,
                    "Total Repayment": None,
                    "Net Interest Cost": None,
                    "Annual IRR (%)": None,
                    "Effective Interest (%)": None,
                }
            )

    df = pd.DataFrame(results)
    return df


# Example usage with current scenario
print("=" * 80)
print("BID AMOUNT SCENARIO ANALYSIS")
print("=" * 80)

scenario_results = analyze_bid_scenarios(
    total_installments=total_installments,
    current_installment_number=current_installment_number,
    full_chit_value=full_chit_value,
    chit_frequency_per_year=chit_frequency_per_year,
    previous_installments=previous_installments,
    bid_amounts_to_test=range(50000, 210000, 10000),
    winner_installment_amount=50000,
)

# Display formatted results
pd.options.display.float_format = "{:,.2f}".format
print(scenario_results.to_string(index=False))

# Find optimal bid (lowest IRR)
optimal_row = scenario_results.loc[scenario_results["Annual IRR (%)"].idxmin()]
print("\n" + "=" * 80)
print("OPTIMAL BID STRATEGY")
print("=" * 80)
print(f"Bid Amount: ₹{optimal_row['Bid Amount']:,.0f}")
print(f"Prize Received: ₹{optimal_row['Prize Received']:,.0f}")
print(f"Annual IRR: {optimal_row['Annual IRR (%)']:.2f}%")
print(f"Net Interest Cost: ₹{optimal_row['Net Interest Cost']:,.0f}")


BID AMOUNT SCENARIO ANALYSIS
 Bid Amount  Prize Received  Total Repayment  Net Interest Cost  Annual IRR (%)  Effective Interest (%)
      50000          650000           665000              15000            1.33                    2.31
      60000          640000           665000              25000            2.27                    3.91
      70000          630000           665000              35000            3.26                    5.56
      80000          620000           665000              45000            4.31                    7.26
      90000          610000           665000              55000            5.42                    9.02
     100000          600000           665000              65000            6.60                   10.83
     110000          590000           665000              75000            7.85                   12.71
     120000          580000           665000              85000            9.20                   14.66
     130000          570000        